## Regression Tables via `pf.etable()` and the `Stargazer` package

To produce regression tables, we have two options: pyfixest's internal `etable()` function and the [Stargazer](https://github.com/StatsReporting/stargazer) Python package.

In [4]:
import pyfixest as pf

%load_ext autoreload
%autoreload 2

data = pf.get_data()

fit1 = pf.feols("Y ~ X1 | f1", data = data)
fit2 = pf.feols("Y ~ X1 | f1 + f2", data = data)
fit3 = pf.feols("Y ~ X1 + X2 | f1", data = data)
fit4 = pf.feols("Y ~ X1 + X2 | f1 + f2", data = data)
fit5 = pf.feols("Y ~ X1 *X2 | f1 + f2", data = data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


IndentationError: unexpected indent (summarize.py, line 261)

## `pf.etable()`

We can now compare all regression models via the `etable()` function: 

In [5]:
pf.etable([fit1, fit2, fit3, fit4, fit5])

NameError: name 'pf' is not defined

`etable` allows us to do a few things out of the box. For example, we can only keep the variables that we'd like, which keeps all variables that fit the provided regex match. 

In [ ]:
pf.etable([fit1, fit2, fit3, fit4, fit5], keep = "X1")

,est1,est2,est3,est4,est5
depvar,Y,Y,Y,Y,Y
X1,-0.949*** (0.069),-0.919*** (0.065),-0.950*** (0.067),-0.924*** (0.061),-0.924*** (0.061)
X1:X2,,,,,0.011 (0.018)
f2,-,x,-,x,x
f1,x,x,x,x,x
R2,0.437,0.609,0.489,0.659,0.659
S.E. type,by: f1,by: f1,by: f1,by: f1,by: f1
Observations,997,997,997,997,997


We can use the `exact_match` argument to select a specific set of variables: 

In [ ]:
pf.etable([fit1, fit2, fit3, fit4, fit5], keep = ["X1", "X2"], exact_match = True)

,est1,est2,est3,est4,est5
depvar,Y,Y,Y,Y,Y
X1,-0.949*** (0.069),-0.919*** (0.065),-0.950*** (0.067),-0.924*** (0.061),-0.924*** (0.061)
X2,,,-0.174*** (0.018),-0.174*** (0.015),-0.185*** (0.025)
f2,-,x,-,x,x
f1,x,x,x,x,x
R2,0.437,0.609,0.489,0.659,0.659
S.E. type,by: f1,by: f1,by: f1,by: f1,by: f1
Observations,997,997,997,997,997


We can also easily **drop** variables via the `drop` argument: 

In [ ]:
pf.etable([fit1, fit2, fit3, fit4, fit5], drop = ["X1"])

,est1,est2,est3,est4,est5
depvar,Y,Y,Y,Y,Y
X2,,,-0.174*** (0.018),-0.174*** (0.015),-0.185*** (0.025)
f2,-,x,-,x,x
f1,x,x,x,x,x
R2,0.437,0.609,0.489,0.659,0.659
S.E. type,by: f1,by: f1,by: f1,by: f1,by: f1
Observations,997,997,997,997,997


By default, `pf.etable()` reports **standard errors**. But we can also ask to output p-values or confidence intervals.

Additionally, we can also overwrite the defaults for the reported significance levels and control the rounding of results via the 
`signif_code` and `digits` function arguments: 

In [ ]:
pf.etable([fit1, fit2, fit3, fit4, fit5], signif_code=[0.01, 0.05, 0.1], digits = 5)

,est1,est2,est3,est4,est5
depvar,Y,Y,Y,Y,Y
X1,-0.94944*** (0.06886),-0.91925*** (0.06539),-0.94953*** (0.06652),-0.92405*** (0.06093),-0.92417*** (0.06094)
X2,,,-0.17423*** (0.01840),-0.17411*** (0.01461),-0.18550*** (0.02516)
X1:X2,,,,,0.01057 (0.01818)
f2,-,x,-,x,x
f1,x,x,x,x,x
R2,0.43708,0.60903,0.48899,0.65904,0.65916
S.E. type,by: f1,by: f1,by: f1,by: f1,by: f1
Observations,997,997,997,997,997


By default, `pf.etable()` returns a data frame, but you can also opt to output latex and markdown via the `type` argument.

In [ ]:
pf.etable([fit1, fit2, fit3, fit4, fit5], signif_code=[0.01, 0.05, 0.1], digits = 5, type = "tex")

NameError: name 'pf' is not defined

## Via `Stargazer`

tba